### Copy Enterprise Geodatabase to New File Geodatabase

##### This notebook copies contents of enterprise geodataase to a new file geodatabase.

* Please make sure to use a correct directory where your enterprise geodatabases are stored.
1. Define Enterprise geodatabase path and names.
    @ This code automatically chooses multiple sde directories and run the code for each SDE
2. Create a file geodatabase (target GDB)
3. Copy SDE to the newly created FGDB

In [1]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy
from pathlib import Path

arcpy.env.overwriteOutput = True

In [2]:
# Obtain all the SDE
home = str(Path.home())
projectFolder = os.path.join(home,'OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/BACKUP')
listDir = os.listdir(projectFolder)

sdeListArray = []
for e in listDir:
    if '.sde' in e:
        sdeListArray.append(e)
sdeListArray

['DOTr_Planning.sde', 'L1CE.sde', 'MMSP.sde', 'NSCR-Ex.sde', 'NSCR.sde']

In [3]:
sdeListArray[-1]

'NSCR.sde'

In [3]:
# Loop for each .sde, create a new FGDB and backup to the new fgdb
fgdb_path = os.path.join(home,'Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/13-File_Geodatabase')

for sde in sdeListArray:
    
    ## Make sure to define workspace for each sde
    sde_path = os.path.join(projectFolder, sde)
    arcpy.env.workspace = sde_path
    
    # Create a new File Geodatabase
    fgdb_name = time.strftime("%Y%m%d") + "_" + sde.replace('.sde','')
    arcpy.management.CreateFileGDB(fgdb_path, fgdb_name)
    
    # targetGDB
    targetGDB = os.path.join(fgdb_path, fgdb_name + ".gdb") 
    
    # Copy stand-alone Feature Classes
    featureClasses = arcpy.ListFeatureClasses()
    for fc in featureClasses:
        print('Feature Class: {}'.format(fc))
        targetName = fc.split('.')[-1]
        targetPath = os.path.join(targetGDB, targetName)
        
        arcpy.CopyFeatures_management(fc, targetPath)
    
    # Copy tables
    tables = arcpy.ListTables()
    for table in tables:
        print('Feature Table: {}'.format(table))
        
        targetName = table.split('.')[-1]
        print('targetName of Table: {}'.format(table))
        
        targetPath = os.path.join(targetGDB, targetName)
        
        arcpy.Copy_management(table, targetPath)
        
    # Copy Feature Classes insdie Feature Datasets
    datasetList = arcpy.ListDatasets(feature_type='feature')
    for ds in datasetList:
        print('Feature Dataset: {}'.format(ds))      
        
        # Obtain spatial refernce of a feature dataset
        desc = arcpy.Describe(ds)
        sr = desc.spatialReference
        featureDatasetName = ds.split('.')[-1] # e.g., Alignment
        
        # If the name of feature dataset starts with numbers, we need to remove it
        try:
            reg = re.compile(r'\d')
            m = reg.search(featureDatasetName).group()
            featureDatasetName = reg.sub(r'',featureDatasetName)
        except:
            featureDasetName = featureDatasetName
        
        # Create a feature dataset
        arcpy.CreateFeatureDataset_management(targetGDB, featureDatasetName, sr)
        print("Feature Dataset: {}".format(featureDatasetName))
        
        for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
            targetName = fc.split('.')[-1]
            targetPath = os.path.join(targetGDB, featureDatasetName, targetName)
            
            arcpy.CopyFeatures_management(fc, targetPath)

Feature Dataset: L1CE.L1CEUSER.L1CE_Land_Acquisition
Feature Dataset: LCE_Land_Acquisition
Feature Dataset: L1CE.L1CEUSER.L1CE_Viaduct
Feature Dataset: LCE_Viaduct
Feature Class: MMSP.MMSPUSER.MMSP_Land_test
Feature Class: MMSP.MMSPUSER.MMSP_Tree_test
Feature Class: MMSP.MMSPUSER.MMSP_Affected_Structures_Test
Feature Class: MMSP.MMSPUSER.CP104_Existing_Utility_Point_PRS92
Feature Class: MMSP.MMSPUSER.CP104_Existing_Utility_Line_PRS92
Feature Class: MMSP.MMSPUSER.CONSTRUCTION_BOUNDARY_NEW
Feature Table: MMSP.MMSPUSER.MMSP_TBM_Assembly_Progress
targetName of Table: MMSP.MMSPUSER.MMSP_TBM_Assembly_Progress
Feature Table: MMSP.MMSPUSER.Site_Photo__ATTACH
targetName of Table: MMSP.MMSPUSER.Site_Photo__ATTACH
Feature Dataset: MMSP.MMSPUSER.Alignment
Feature Dataset: Alignment
Feature Dataset: MMSP.MMSPUSER.Civil_Construction
Feature Dataset: Civil_Construction


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failure to access the DBMS server [[Microsoft][ODBC Driver 17 for SQL Server]Communication link failure] [MMSP.sde.GDB_Items]
Failure to access the DBMS server [08S01:[Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.
] [MMSP.MMSPUSER.MMSP_Buildings_Depot]
Failed to execute (CopyFeatures).


In [55]:
try:
    reg = re.compile(r'\d')
    m = reg.search(test).group()
    test = reg.sub(r'', test)
    print(test)
except:
    test = 'Error'
    print(test)

_Land_Acquisition
